<a href="https://colab.research.google.com/github/elliecheshire/Stop_and_Search/blob/main/Stop_Search_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount drive to allow access to files
from google.colab import drive; drive.mount('/content/drive')
import pandas as pd

# read in .xlsx files into data frames
# these .xlsx files contain all stop and search data from police forces across Endland and Wales
# data is split into 2 files based on years

data07_20 = pd.read_excel('/content/drive/MyDrive/programming_freeform_assessment/stop_search_07_20.xlsx')

data21_23 = pd.read_excel('/content/drive/MyDrive/programming_freeform_assessment/stop_search_21_23.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,financial_year,financial_year_quarter,geocode,police_force_area,region,legislation,reason_for_search,outcome,link,self_defined_ethnicity,self_defined_ethnicity_group,officer_defined_ethnicity,combined_ethnicity,sex,age_group,number_of_searches
0,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Nothing found,Pakistani or British Pakistani,Asian or Asian British,NaN,Asian or Asian British,Male,25-29,1
1,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Linked,Black or Black British African,Black or Black British,NaN,Black or Black British,Male,30 or over,1
2,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Linked,Not Stated,Not Stated,Other,Mixed or Other,Male,18-24,1
3,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Not linked,Not Stated,Not Stated,White,White,Male,30 or over,1
4,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Nothing found,Not Stated,Not Stated,White,White,Male,30 or over,1


In [4]:
data07_20.head()
data21_23.head()

,financial_year,financial_year_quarter,geocode,police_force_area,region,legislation,reason_for_search,outcome,link,self_defined_ethnicity,self_defined_ethnicity_group,officer_defined_ethnicity,combined_ethnicity,sex,age_group,number_of_searches
0,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Nothing found,Pakistani or British Pakistani,Asian or Asian British,NaN,Asian or Asian British,Male,25-29,1
1,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Linked,Black or Black British African,Black or Black British,NaN,Black or Black British,Male,30 or over,1
2,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Linked,Not Stated,Not Stated,Other,Mixed or Other,Male,18-24,1
3,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Not linked,Not Stated,Not Stated,White,White,Male,30 or over,1
4,2020/21,1,E23000036,Avon & Somerset,South West,Section 1 (PACE),Criminal Damage,Arrest,Nothing found,Not Stated,Not Stated,White,White,Male,30 or over,1


fatal: not a git repository (or any of the parent directories): .git
